# 施設共用課題Excelファイルから利用ユーザーの予定をインポート

In [32]:
import pandas as pd
import datetime
# NOTE*xlrdのインストールが必要
# ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 
# for Excel support Use pip or conda to install xlrd.

# input_book = pd.ExcelFile('施設供用課題データベース(2023A).xls')
# print(input_book)
# input_sheet_names = input_book.sheet_names
# input_sheet_df = input_book.parse(input_sheet_names[0])

input_sheet_df = pd.read_excel('施設供用課題データベース(2023A).xls', sheet_name=0, usecols=(8, 9, 17, 21, 22))
print(input_sheet_df)

print(input_sheet_df.values[0][4])
str_datetime = input_sheet_df.values[0][4].strftime('%Y-%m-%d %H:%M')
print(str_datetime)


        BL            利用装置 機構\n担当者            利用開始日時（予定）            利用終了日時（予定）
0   BL14B1  エネルギー分散型XAFS装置   松村　大樹   2023-07-17 10:00:00   2023-07-21 10:00:00
1   BL14B1  エネルギー分散型XAFS装置   松村　大樹   2023-06-21 00:00:00   2023-06-27 00:00:00
2   BL14B1  エネルギー分散型XAFS装置   松村　大樹   2023-04-14 00:00:00   2023-04-18 00:00:00
3   BL14B1  エネルギー分散型XAFS装置   松村　大樹   2023-05-18 00:00:00   2023-05-22 00:00:00
4   BL22XU        XAFS測定装置   塩飽　秀啓   2023-06-27 00:00:00   2023-06-29 00:00:00
5   BL22XU        XAFS測定装置    矢板　毅   2023-06-12 00:00:00   2023-06-14 00:00:00
6   BL22XU   応力・イメージング測定装置   菖蒲　敬久   2023-06-02 00:00:00   2023-06-04 00:00:00
7   BL22XU         κ型X線回折計   田村　和久   2023-06-20 00:00:00   2023-06-23 00:00:00
8   BL22XU   応力・イメージング測定装置   菖蒲　敬久   2023-05-31 00:00:00   2023-06-02 00:00:00
9   BL22XU         κ型X線回折計   田村　和久   2023-05-11 00:00:00   2023-05-14 00:00:00
10  BL22XU        XAFS測定装置    小林　徹   2023/6/7\n2023/6/23   2023/6/9\n2023/6/24
11  BL23SU    表面化学実験ステーション   吉越　章隆  2023/5/28\n2023/

- 予定をOutlookに登録する

In [51]:
import pandas as pd
import win32com.client
import datetime
import re

SCHEDULE_FILE = '施設供用課題データベース(2023A).xls'

def read_excel_schedule(excel_filename):
    input_sheet_df = pd.read_excel(excel_filename, sheet_name=0, usecols=(8, 9, 17, 21, 22))
    return input_sheet_df

def add_outlook_schedule(excel_filename):
    input_sheet_df = read_excel_schedule(excel_filename)
    # Outlookの予定表は1　（0:メール)
    APPOINTMENT_ITEM = 1
    outlook = win32com.client.Dispatch("Outlook.Application")
    # mapi = outlook.GetNamespace("MAPI")

    for values in input_sheet_df.values:
        print(values)
        
        beam_line, instrument, instrument_pic, start_date, end_date =  values
        if isinstance(start_date, datetime.datetime):
            item = outlook.CreateItem(APPOINTMENT_ITEM)
            # NOTE*datetime型をわたすより文字列が安定している
            # フォーマットは'%Y-%m-%d %H:%M'
            # Datetime.strptimeは日付が変になる
            item.Start =  start_date.strftime('%Y-%m-%d %H:%M')
            item.End = end_date.strftime('%Y-%m-%d %H:%M')
            item.Subject = instrument
            item.Location = beam_line
            item.Body = f'担当者：{instrument_pic}'
            # 分類を決定
            item.Categories = 'ビームタイム'
            item.Save()
        elif isinstance(start_date, str):
            item = outlook.CreateItem(APPOINTMENT_ITEM)
            start_date_list = start_date.split('\n')
            end_date_list = end_date.split('\n')            
            for i in range(len(start_date_list)):
                item = outlook.CreateItem(APPOINTMENT_ITEM)
                splited_start_date = start_date_list[i].split('/')
                splited_end_date = end_date_list[i].split('/')
                print(f'start {splited_start_date[0]}-{splited_start_date[1]}-{splited_start_date[2]} 10:00')
                print(f'end {splited_end_date[0]}-{splited_end_date[1]}-{splited_end_date[2]} 10:00')
                item.Start = f'{splited_start_date[0]}-{splited_start_date[1]}-{splited_start_date[2]} 10:00'
                item.End = f'{splited_end_date[0]}-{splited_end_date[1]}-{splited_end_date[2]} 10:00'
                item.Subject = instrument
                item.Location = beam_line
                item.Body = f'担当者：{instrument_pic}'
                # 分類を決定
                item.Categories = 'ビームタイム'
                item.Save()


add_outlook_schedule(SCHEDULE_FILE)


    


['BL14B1' 'エネルギー分散型XAFS装置' '松村\u3000大樹'
 datetime.datetime(2023, 7, 17, 10, 0)
 datetime.datetime(2023, 7, 21, 10, 0)]
['BL14B1' 'エネルギー分散型XAFS装置' '松村\u3000大樹'
 datetime.datetime(2023, 6, 21, 0, 0) datetime.datetime(2023, 6, 27, 0, 0)]
['BL14B1' 'エネルギー分散型XAFS装置' '松村\u3000大樹'
 datetime.datetime(2023, 4, 14, 0, 0) datetime.datetime(2023, 4, 18, 0, 0)]
['BL14B1' 'エネルギー分散型XAFS装置' '松村\u3000大樹'
 datetime.datetime(2023, 5, 18, 0, 0) datetime.datetime(2023, 5, 22, 0, 0)]
['BL22XU' 'XAFS測定装置' '塩飽\u3000秀啓' datetime.datetime(2023, 6, 27, 0, 0)
 datetime.datetime(2023, 6, 29, 0, 0)]
['BL22XU' 'XAFS測定装置' '矢板\u3000毅' datetime.datetime(2023, 6, 12, 0, 0)
 datetime.datetime(2023, 6, 14, 0, 0)]
['BL22XU' '応力・イメージング測定装置' '菖蒲\u3000敬久' datetime.datetime(2023, 6, 2, 0, 0)
 datetime.datetime(2023, 6, 4, 0, 0)]
['BL22XU' 'κ型X線回折計' '田村\u3000和久' datetime.datetime(2023, 6, 20, 0, 0)
 datetime.datetime(2023, 6, 23, 0, 0)]
['BL22XU' '応力・イメージング測定装置' '菖蒲\u3000敬久'
 datetime.datetime(2023, 5, 31, 0, 0) datetime.dateti

In [42]:
import re
a = '2023/5/28'
re.search('(\d*)/', a).groups()
group = a.split('/')
f'{group[0]}-{group[1]}-{group[2]} 10:00'



'2023-5-28 10:00'